In [1]:
# !pip install pandasai


In [1]:
pip show pandasai

Name: pandasai
Version: 2.3.0
Summary: Chat with your database (SQL, CSV, pandas, polars, mongodb, noSQL, etc). PandasAI makes data analysis conversational using LLMs (GPT 3.5 / 4, Anthropic, VertexAI) and RAG.
Home-page: 
Author: Gabriele Venturi
Author-email: 
License: MIT
Location: C:\Users\mayumadh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: astor, duckdb, faker, jinja2, matplotlib, openai, pandas, pillow, pydantic, python-dotenv, requests, scipy, sqlalchemy, sqlglot
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [4]:
df = pd.read_excel('Sample Data_Copilot.xlsx')

In [35]:
def get_csv_schema(df) -> str:
    """
    Generate schema details from the CSV.
    """
    # Fetching column names and their types
    schema_details = "The CSV contains the following columns:\n"
    # Iterating over the columns and their types
    for column in df.columns:
        dtype = df[column].dtype
        schema_details += f"- Column: *{column}*, Data Type: *{dtype}*\n"
        if dtype == object:
            schema_details += f"Unique Values: {df[column].unique()}"
    # Highlighting important columns (e.g., month columns and dimensions)
    month_columns = [
        col
        for col in df.columns
        if col
        in [
            "Jan",
            "Feb",
            "Mar",
            "Apr",
            "May",
            "Jun",
            "Jul",
            "Aug",
            "Sep",
            "Oct",
            "Nov",
            "Dec",
        ]
    ]
    dimension_columns = [col for col in df.columns if col not in month_columns]
    schema_details += "\n=== Special Columns ===\n"
    schema_details += f"Dimensions: {', '.join(dimension_columns)}\n"

    return schema_details

In [36]:
print(df.shape)
print(df.info())

(2384, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384 entries, 0 to 2383
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Cg_Normalized_Vendor          2384 non-null   object        
 1   PO_L1                         2384 non-null   object        
 2   PO_L2                         2384 non-null   object        
 3   PO_L3                         2384 non-null   object        
 4   PO_L4                         2384 non-null   object        
 5   G/L Acct Long Text            2384 non-null   object        
 6   Company Code                  2384 non-null   object        
 7   Country Short                 2384 non-null   object        
 8   Document number               2384 non-null   object        
 9   Purchasing Document           2384 non-null   object        
 10  Posting Date                  2384 non-null   datetime64[ns]
 11  Payment terms      

In [37]:
schema = get_csv_schema(df)
schema

"The CSV contains the following columns:\n- Column: *Cg_Normalized_Vendor*, Data Type: *object*\nUnique Values: ['HAUPTZOLLAMT GIESSEN' 'HAEMONETICS CORP'\n 'PERSONALVORSORGESTIFTUNG DER CSL BEHRING AG' 'FIDELITY FPRS'\n 'JONES LANG LASALLE AMERICAS INC' 'ZOLLAMT MARBURG' 'CDW LLC'\n 'UFCW NATIONAL HEALTH AND WELFARE FUND'\n 'STEUERVERWALTUNG DES KANTONS BERN ABTEILUNG QUELLENSTEUERN'\n 'BEAR CONSTRUCTION' 'PENSIONSKASSE DMITARBEITE' 'EWB ENERGIE WASSER BERN'\n 'PATHEON UK LTD' 'HEMASOURCE INC' 'HOF SONDERANLAGENBAU' 'SARTORIUS AG'\n 'Not Available' 'PHARMASERV GMBH' 'CSL LTD' 'VMWARE INTERNATIONAL LTD'\n 'BARKER CONTRACTING INC' 'RAMBOLL AMERICAS INTEGRATED SOLUTIO AMERICA'\n 'PATHEON MANUFACTURING SERVICES LLC' 'PAYMENT SLIP CONSUMPTION TAX'\n 'CSL BEHRING' 'WHEELS INC' 'ROCHE DIAGNOSTICS CORP' 'PALL CORP'\n 'MITIE WASTE AND ENVIRONMENTAL' 'AHEAD INC' 'NWR INC'\n 'OTTO BUILDING TECHNOLOGIES GMBH' 'MATTCON GENERAL CONTRACTORS'\n 'SPOTLESS FACILITY SERVICES PTY LTD' 'ABBOTT' 'JAG JACOB

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384 entries, 0 to 2383
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Cg_Normalized_Vendor          2384 non-null   object        
 1   PO_L1                         2384 non-null   object        
 2   PO_L2                         2384 non-null   object        
 3   PO_L3                         2384 non-null   object        
 4   PO_L4                         2384 non-null   object        
 5   G/L Acct Long Text            2384 non-null   object        
 6   Company Code                  2384 non-null   object        
 7   Country Short                 2384 non-null   object        
 8   Document number               2384 non-null   object        
 9   Purchasing Document           2384 non-null   object        
 10  Posting Date                  2384 non-null   datetime64[ns]
 11  Payment terms                 

In [39]:
import re
from dotenv import load_dotenv
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
import os 

def init_openai_client() -> AzureChatOpenAI:
    load_dotenv() 

    endpoint = os.getenv("AZURE_ENDPOINT")
    key = os.getenv("AZURE_API_KEY")
    api_version = os.getenv("AZURE_API_VERSION")
    azure_deployment = os.getenv("AZURE_DEPLOYMENT")
    # httpx_client = httpx.Client(http2=True, verify=False)

    return AzureChatOpenAI(
        api_key=key,
        api_version=api_version,
        azure_endpoint=endpoint,
        azure_deployment=azure_deployment,
        # http_client=httpx_client
    )

In [40]:
client = init_openai_client()

In [ ]:
df_info = get_csv_schema(df)
# query = "create a report to show top cg_normalized_vendor by spend"
query= "Create a report to show cg_normalized_vendor with distinct payment terms"
human_msg  = f"\nThis is the pandas Dataframe info : {df_info}. \nDo Not create your own data, assume that this data is already loaded as df and Answer the User query using proper imports and operations in your code. \nI need you to write a code for this questions. Question: *{query}*"



In [42]:
human_msg

"\nThis is the pandas Dataframe info : The CSV contains the following columns:\n- Column: *Cg_Normalized_Vendor*, Data Type: *object*\nUnique Values: ['HAUPTZOLLAMT GIESSEN' 'HAEMONETICS CORP'\n 'PERSONALVORSORGESTIFTUNG DER CSL BEHRING AG' 'FIDELITY FPRS'\n 'JONES LANG LASALLE AMERICAS INC' 'ZOLLAMT MARBURG' 'CDW LLC'\n 'UFCW NATIONAL HEALTH AND WELFARE FUND'\n 'STEUERVERWALTUNG DES KANTONS BERN ABTEILUNG QUELLENSTEUERN'\n 'BEAR CONSTRUCTION' 'PENSIONSKASSE DMITARBEITE' 'EWB ENERGIE WASSER BERN'\n 'PATHEON UK LTD' 'HEMASOURCE INC' 'HOF SONDERANLAGENBAU' 'SARTORIUS AG'\n 'Not Available' 'PHARMASERV GMBH' 'CSL LTD' 'VMWARE INTERNATIONAL LTD'\n 'BARKER CONTRACTING INC' 'RAMBOLL AMERICAS INTEGRATED SOLUTIO AMERICA'\n 'PATHEON MANUFACTURING SERVICES LLC' 'PAYMENT SLIP CONSUMPTION TAX'\n 'CSL BEHRING' 'WHEELS INC' 'ROCHE DIAGNOSTICS CORP' 'PALL CORP'\n 'MITIE WASTE AND ENVIRONMENTAL' 'AHEAD INC' 'NWR INC'\n 'OTTO BUILDING TECHNOLOGIES GMBH' 'MATTCON GENERAL CONTRACTORS'\n 'SPOTLESS FACILITY

In [43]:
ai_response = client.invoke(human_msg)
ai_response

AIMessage(content="To create a report that shows each `Cg_Normalized_Vendor` along with its distinct payment terms, you can use the Pandas library in Python. The following code snippet achieves this by grouping the data by `Cg_Normalized_Vendor` and aggregating the distinct payment terms for each vendor.\n\nHere's the code to generate the report:\n\n```python\nimport pandas as pd\n\n# Assuming df is already loaded as your DataFrame\n\n# Group by 'Cg_Normalized_Vendor' and aggregate distinct 'Payment terms'\nreport = df.groupby('Cg_Normalized_Vendor')['Payment terms'].unique().reset_index()\n\n# Rename the columns for clarity\nreport.columns = ['Cg_Normalized_Vendor', 'Distinct Payment Terms']\n\n# Display the report\nprint(report)\n```\n\n### Explanation:\n1. **Group By**: The `groupby` method is used to group the DataFrame by the `Cg_Normalized_Vendor` column.\n2. **Aggregation**: The `unique` function is applied to the `Payment terms` column to get distinct payment terms for each ven

In [44]:
print(ai_response.content)

To create a report that shows each `Cg_Normalized_Vendor` along with its distinct payment terms, you can use the Pandas library in Python. The following code snippet achieves this by grouping the data by `Cg_Normalized_Vendor` and aggregating the distinct payment terms for each vendor.

Here's the code to generate the report:

```python
import pandas as pd

# Assuming df is already loaded as your DataFrame

# Group by 'Cg_Normalized_Vendor' and aggregate distinct 'Payment terms'
report = df.groupby('Cg_Normalized_Vendor')['Payment terms'].unique().reset_index()

# Rename the columns for clarity
report.columns = ['Cg_Normalized_Vendor', 'Distinct Payment Terms']

# Display the report
print(report)
```

### Explanation:
1. **Group By**: The `groupby` method is used to group the DataFrame by the `Cg_Normalized_Vendor` column.
2. **Aggregation**: The `unique` function is applied to the `Payment terms` column to get distinct payment terms for each vendor.
3. **Reset Index**: The `reset_inde

In [46]:
def extract_python_code(text):
    match = re.search(r"```python\n(.*?)\n```", text, re.DOTALL)
    return match.group(1) if match else ""

In [47]:
code = extract_python_code(ai_response.content)

In [48]:
print(code)

import pandas as pd

# Assuming df is already loaded as your DataFrame

# Group by 'Cg_Normalized_Vendor' and aggregate distinct 'Payment terms'
report = df.groupby('Cg_Normalized_Vendor')['Payment terms'].unique().reset_index()

# Rename the columns for clarity
report.columns = ['Cg_Normalized_Vendor', 'Distinct Payment Terms']

# Display the report
print(report)


In [49]:
# create a report to show top cg_normalized_vendor by spend 

import pandas as pd

vendor_spend = df.groupby('Cg_Normalized_Vendor')['Spend'].sum()

# Sort the vendors by spend in descending order
top_vendors_by_spend = vendor_spend.sort_values(ascending=False)

result_df = top_vendors_by_spend.map(lambda x: f"{x:.2f}" if isinstance(x, (int, float)) else x)

# Display the top vendors by spend
top_vendors_by_spend

Cg_Normalized_Vendor
HAUPTZOLLAMT GIESSEN                           1.099890e+08
ZOLLAMT MARBURG                                6.957429e+07
FIDELITY FPRS                                  2.267974e+07
PERSONALVORSORGESTIFTUNG DER CSL BEHRING AG    1.288088e+07
HAEMONETICS CORP                               1.189783e+07
                                                   ...     
FOUR LEAF SUPPLY CORP                          9.288690e+03
PRICEWATERHOUSECOOPERS                         9.193870e+03
STAWIN AG                                      8.987480e+03
IFB                                            8.794270e+03
EUROROOF GMBH                                  8.759730e+03
Name: Spend, Length: 358, dtype: float64

In [50]:
result_df

Cg_Normalized_Vendor
HAUPTZOLLAMT GIESSEN                           109989042.92
ZOLLAMT MARBURG                                 69574288.27
FIDELITY FPRS                                   22679739.89
PERSONALVORSORGESTIFTUNG DER CSL BEHRING AG     12880881.43
HAEMONETICS CORP                                11897826.51
                                                   ...     
FOUR LEAF SUPPLY CORP                               9288.69
PRICEWATERHOUSECOOPERS                              9193.87
STAWIN AG                                           8987.48
IFB                                                 8794.27
EUROROOF GMBH                                       8759.73
Name: Spend, Length: 358, dtype: object

In [51]:
import pandas as pd

# Assuming df is your already loaded DataFrame

# Create the report showing distinct payment terms for each vendor
report = df.groupby('Cg_Normalized_Vendor')['Payment terms'].unique().reset_index()

# Rename the columns for clarity
report.columns = ['Cg_Normalized_Vendor', 'Distinct_Payment_Terms']

# Display the report
print(report)

                           Cg_Normalized_Vendor Distinct_Payment_Terms
0                                        ABBOTT           [T105, T045]
1                           ABSG CONSULTING INC                 [T060]
2                          AC ISOLIERUNGEN GMBH                 [T060]
3                                 ACCENTURE PLC                 [T045]
4                                 ACOEM USA INC                 [T060]
..                                          ...                    ...
353                                   ZETA GMBH           [T030, T060]
354  ZIG ZAG SOLUCIONES CREATIVAS INTEGRALES SL                 [T060]
355                             ZIMMERMANN GMBH     [T060, T030, T010]
356                   ZIMMERMANN MESSTECHNIK AG                 [T030]
357                             ZOLLAMT MARBURG                 [T060]

[358 rows x 2 columns]


In [ ]:
#**Important Notes:**
        #  * Always use `df` to refer to the current year's data.
        #  * **Do not include any code to load the DataFrame.** They are already loaded and available in the environment.
        #  * Your final Python code **must** have the resulting pandas DataFrame as the last variable in the code block. This DataFrame will be the output of your analysis.

        #  **Output:**
        #  Generate complete and executable pandas code that performs the requested data analysis on `primary_df`. The code should handle necessary operations and ensure the final result is a pandas DataFrame.